In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import talib                     
import pynance as py             
import quantstats as qs 
import glob
import os


# Make plots a bit nicer
plt.style.use("seaborn-v0_8")
plt.rcParams["figure.figsize"] = (12, 6)


In [5]:

df = pd.read_csv("../data/AAPL.csv")
df.head()


,Date,Close,High,Low,Open,Volume
0,2009-01-02,2.721686,2.730385,2.554037,2.575630,746015200
1,2009-01-05,2.836553,2.884539,2.780469,2.794266,1181608400
2,2009-01-06,2.789767,2.914229,2.770872,2.877641,1289310400
3,2009-01-07,2.729484,2.774170,2.706990,2.753477,753048800
4,2009-01-08,2.780169,2.793666,2.700393,2.712090,673500800


In [6]:

df_news = pd.read_csv("../data/raw_analyst_ratings.csv")
print(df_news['stock'].unique()[:30])
print("Total unique tickers:", df_news['stock'].nunique())

['A' 'AA' 'AAC' 'AADR' 'AAL' 'AAMC' 'AAME' 'AAN' 'AAOI' 'AAON' 'AAP'
 'AAPL' 'AAU' 'AAV' 'AAVL' 'AAWW' 'AAXJ' 'AB' 'ABAC' 'ABAX' 'ABB' 'ABBV'
 'ABC' 'ABCB' 'ABCD' 'ABCO' 'ABCW' 'ABDC' 'ABEV' 'ABG']
Total unique tickers: 6204


In [9]:
files = glob.glob("../data/*.csv")
price_files = [f for f in files if "raw_analyst_ratings" not in f]
price_files

['../data\\AAPL.csv',
 '../data\\AMZN.csv',
 '../data\\GOOG.csv',
 '../data\\META.csv',
 '../data\\MSFT.csv',
 '../data\\NVDA.csv']

In [10]:
dataframes = []

for file in price_files:
    df = pd.read_csv(file)
    ticker = os.path.basename(file).replace(".csv", "")
    df["Ticker"] = ticker
    dataframes.append(df)


In [11]:
all_prices = pd.concat(dataframes, ignore_index=True)


In [26]:
ratings = pd.read_csv("../data/raw_analyst_ratings.csv")

In [27]:
ratings["date"] = pd.to_datetime(ratings["date"], errors="coerce", utc=True)
ratings["stock"] = ratings["stock"].str.upper()
ratings[ratings["date"].isna()]
ratings['date'].isna().sum()



np.int64(1351341)

In [32]:
ratings["date"].head(20).tolist()


[Timestamp('2020-06-05 14:30:54+0000', tz='UTC'),
 Timestamp('2020-06-03 14:45:20+0000', tz='UTC'),
 Timestamp('2020-05-26 08:30:07+0000', tz='UTC'),
 Timestamp('2020-05-22 16:45:06+0000', tz='UTC'),
 Timestamp('2020-05-22 15:38:59+0000', tz='UTC'),
 Timestamp('2020-05-22 15:23:25+0000', tz='UTC'),
 Timestamp('2020-05-22 13:36:20+0000', tz='UTC'),
 Timestamp('2020-05-22 13:07:04+0000', tz='UTC'),
 Timestamp('2020-05-22 12:37:59+0000', tz='UTC'),
 Timestamp('2020-05-22 12:06:17+0000', tz='UTC'),
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT]

In [30]:
ratings["date"].dtype


datetime64[ns, UTC]

In [34]:
ratings_raw = pd.read_csv("../data/raw_analyst_ratings.csv", dtype={"date": "string"})


In [25]:
df_raw = pd.read_csv("../data/raw_analyst_ratings.csv", dtype={"date": "string"})

# Try converting
parsed = pd.to_datetime(df_raw["date"], errors="coerce")

# Get bad rows correctly
bad_raw = df_raw[parsed.isna()]

bad_raw.head()
bad_raw.sample(10)
bad_raw["date"].unique()[:50]


<StringArray>
['2020-05-22 00:00:00', '2020-05-21 00:00:00', '2020-05-18 00:00:00',
 '2020-05-16 00:00:00', '2020-05-15 00:00:00', '2020-05-08 00:00:00',
 '2020-05-05 00:00:00', '2020-05-01 00:00:00', '2020-04-28 00:00:00',
 '2020-04-23 00:00:00', '2020-04-22 00:00:00', '2020-04-14 00:00:00',
 '2020-04-08 00:00:00', '2020-04-06 00:00:00', '2020-04-02 00:00:00',
 '2020-04-01 00:00:00', '2020-03-31 00:00:00', '2020-03-30 00:00:00',
 '2020-03-27 00:00:00', '2020-03-26 00:00:00', '2020-03-20 00:00:00',
 '2020-03-16 00:00:00', '2020-03-13 00:00:00', '2020-03-12 00:00:00',
 '2020-03-11 00:00:00', '2020-03-03 00:00:00', '2020-03-02 00:00:00',
 '2020-02-24 00:00:00', '2020-02-19 00:00:00', '2020-02-18 00:00:00',
 '2020-02-14 00:00:00', '2020-02-06 00:00:00', '2020-02-05 00:00:00',
 '2020-01-30 00:00:00', '2020-01-22 00:00:00', '2020-01-17 00:00:00',
 '2020-01-16 00:00:00', '2020-01-15 00:00:00', '2020-01-10 00:00:00',
 '2020-01-09 00:00:00', '2020-01-08 00:00:00', '2020-01-07 00:00:00',
 '2020

In [31]:
ratings[ratings["date"].str.contains("-", na=False)].head()
ratings[ratings["date"].str.contains("/", na=False)].head()


AttributeError: Can only use .str accessor with string values!

In [33]:
invalid_dates = ratings[ratings["date"].isna()][["date"]]
invalid_dates.head(20)


,date
10,NaT
11,NaT
12,NaT
13,NaT
14,NaT
15,NaT
16,NaT
17,NaT
18,NaT
19,NaT


In [35]:
ratings_raw['date'].head(20).tolist()

['2020-06-05 10:30:54-04:00',
 '2020-06-03 10:45:20-04:00',
 '2020-05-26 04:30:07-04:00',
 '2020-05-22 12:45:06-04:00',
 '2020-05-22 11:38:59-04:00',
 '2020-05-22 11:23:25-04:00',
 '2020-05-22 09:36:20-04:00',
 '2020-05-22 09:07:04-04:00',
 '2020-05-22 08:37:59-04:00',
 '2020-05-22 08:06:17-04:00',
 '2020-05-22 00:00:00',
 '2020-05-22 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-18 00:00:00',
 '2020-05-16 00:00:00',
 '2020-05-15 00:00:00',
 '2020-05-08 00:00:00']

In [36]:
# Convert with errors='coerce' but keep raw
temp = pd.to_datetime(ratings_raw["date"], errors="coerce", utc=True)

# All rows that failed
invalid = ratings_raw[temp.isna()]["date"]

invalid.head(50).tolist()


['2020-05-22 00:00:00',
 '2020-05-22 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-21 00:00:00',
 '2020-05-18 00:00:00',
 '2020-05-16 00:00:00',
 '2020-05-15 00:00:00',
 '2020-05-08 00:00:00',
 '2020-05-05 00:00:00',
 '2020-05-01 00:00:00',
 '2020-04-28 00:00:00',
 '2020-04-23 00:00:00',
 '2020-04-22 00:00:00',
 '2020-04-14 00:00:00',
 '2020-04-08 00:00:00',
 '2020-04-06 00:00:00',
 '2020-04-02 00:00:00',
 '2020-04-01 00:00:00',
 '2020-03-31 00:00:00',
 '2020-03-30 00:00:00',
 '2020-03-30 00:00:00',
 '2020-03-27 00:00:00',
 '2020-03-26 00:00:00',
 '2020-03-26 00:00:00',
 '2020-03-26 00:00:00',
 '2020-03-20 00:00:00',
 '2020-03-16 00:00:00',
 '2020-03-16 00:00:00',
 '2020-03-13 00:00:00',
 '2020-03-12 00:00:00',
 '2020-03-11 00:00:00',
 '2020-03-03 00:00:00',
 '2020-03-02 00:00:00',
 '2020-02-24 00:00:00',
 '2020-02-24 00:00:00',
 '2020-02-24 00:00:00',
 '2020-02-19 00:00:00',
 '2020-02-18 00:00:00',
 '2020-02-18 00:00:00',
 '2020-02-18 00:

In [37]:
ratings_raw["date"].dtype


string[python]

In [38]:
ratings_raw["date"].isna().sum()


np.int64(0)

In [39]:
# Attempt conversion
temp = pd.to_datetime(ratings_raw["date"], errors="coerce")

# Extract only the raw strings that became NaT
invalid_raw = ratings_raw.loc[temp.isna(), "date"]

invalid_raw.value_counts().head(50)


date
2020-03-12 00:00:00    1766
2020-02-27 00:00:00    1292
2020-02-28 00:00:00    1239
2019-08-01 00:00:00    1228
2020-03-06 00:00:00    1147
2018-08-02 00:00:00    1138
2009-08-10 00:00:00    1130
2018-10-30 00:00:00    1129
2018-07-26 00:00:00    1125
2019-07-31 00:00:00    1116
2019-08-07 00:00:00    1114
2020-03-11 00:00:00    1102
2018-11-01 00:00:00    1101
2018-08-08 00:00:00    1089
2020-04-15 00:00:00    1067
2019-11-07 00:00:00    1065
2018-08-01 00:00:00    1062
2020-03-09 00:00:00    1052
2020-01-22 00:00:00    1048
2020-01-21 00:00:00    1046
2019-07-30 00:00:00    1045
2018-10-29 00:00:00    1041
2019-08-05 00:00:00    1032
2011-07-28 00:00:00    1032
2020-03-16 00:00:00    1023
2018-10-24 00:00:00    1015
2020-03-23 00:00:00    1015
2020-02-20 00:00:00    1009
2017-11-02 00:00:00    1007
2018-08-07 00:00:00    1004
2018-07-25 00:00:00    1002
2020-03-10 00:00:00     983
2020-03-17 00:00:00     975
2019-08-08 00:00:00     973
2018-11-08 00:00:00     966
2018-08-09 00:0

In [40]:
ratings_raw["date"].apply(type).value_counts()


date
<class 'str'>    1407328
Name: count, dtype: int64

In [43]:
ratings_raw["date"].str.contains(r"[^\x00-\x7F]", regex=True).sum()


np.int64(0)

In [42]:
ratings_raw.loc[temp.isna(), "date"].head(20).apply(repr).tolist()


["'2020-05-22 00:00:00'",
 "'2020-05-22 00:00:00'",
 "'2020-05-21 00:00:00'",
 "'2020-05-21 00:00:00'",
 "'2020-05-21 00:00:00'",
 "'2020-05-21 00:00:00'",
 "'2020-05-18 00:00:00'",
 "'2020-05-16 00:00:00'",
 "'2020-05-15 00:00:00'",
 "'2020-05-08 00:00:00'",
 "'2020-05-05 00:00:00'",
 "'2020-05-01 00:00:00'",
 "'2020-04-28 00:00:00'",
 "'2020-04-23 00:00:00'",
 "'2020-04-22 00:00:00'",
 "'2020-04-14 00:00:00'",
 "'2020-04-08 00:00:00'",
 "'2020-04-06 00:00:00'",
 "'2020-04-02 00:00:00'",
 "'2020-04-01 00:00:00'"]

In [44]:
ratings_raw["date_clean"] = ratings_raw["date"].str.strip("'\"")


In [45]:
ratings_raw["date_parsed"] = pd.to_datetime(ratings_raw["date_clean"], errors="coerce", utc=True)
ratings_raw["date_parsed"].isna().sum()


np.int64(1351341)

In [46]:
ratings_raw.head()

,Unnamed: 0,headline,url,publisher,date,stock,date_clean,date_parsed
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A,2020-06-05 10:30:54-04:00,2020-06-05 14:30:54+00:00
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A,2020-06-03 10:45:20-04:00,2020-06-03 14:45:20+00:00
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A,2020-05-26 04:30:07-04:00,2020-05-26 08:30:07+00:00
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A,2020-05-22 12:45:06-04:00,2020-05-22 16:45:06+00:00
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A,2020-05-22 11:38:59-04:00,2020-05-22 15:38:59+00:00


In [47]:
ratings_raw["date_parsed"].isna().mean()  # fraction of missing dates
ratings_raw["date_parsed"].notna().sum()  # count of valid dates
ratings_raw["date_parsed"].isna().sum()   # count of invalid/missing dates


np.int64(1351341)

In [48]:
ratings_raw.loc[ratings_raw["date_parsed"].isna(), "date"].value_counts().head(20)


date
2020-03-12 00:00:00    1766
2020-02-27 00:00:00    1292
2020-02-28 00:00:00    1239
2019-08-01 00:00:00    1228
2020-03-06 00:00:00    1147
2018-08-02 00:00:00    1138
2009-08-10 00:00:00    1130
2018-10-30 00:00:00    1129
2018-07-26 00:00:00    1125
2019-07-31 00:00:00    1116
2019-08-07 00:00:00    1114
2020-03-11 00:00:00    1102
2018-11-01 00:00:00    1101
2018-08-08 00:00:00    1089
2020-04-15 00:00:00    1067
2019-11-07 00:00:00    1065
2018-08-01 00:00:00    1062
2020-03-09 00:00:00    1052
2020-01-22 00:00:00    1048
2020-01-21 00:00:00    1046
Name: count, dtype: Int64

In [49]:
ratings_raw["date_parsed"] = pd.to_datetime(
    ratings_raw["date"].str.strip().str.strip("'\""),
    errors="coerce",
    utc=True
)


In [50]:
ratings_raw["date_parsed"].isna().sum()   # should drop drastically
ratings_raw["date_parsed"].head()


0   2020-06-05 14:30:54+00:00
1   2020-06-03 14:45:20+00:00
2   2020-05-26 08:30:07+00:00
3   2020-05-22 16:45:06+00:00
4   2020-05-22 15:38:59+00:00
Name: date_parsed, dtype: datetime64[ns, UTC]

In [51]:
recent_news = ratings_raw[ratings_raw["date_parsed"] >= "2020-01-01"]


In [52]:
ratings_raw["year"] = ratings_raw["date_parsed"].dt.year
ratings_raw["month"] = ratings_raw["date_parsed"].dt.month
ratings_raw["day"] = ratings_raw["date_parsed"].dt.day


In [53]:
ratings_raw.dropna(subset=["date_parsed"], inplace=True)
